# 02 - ANERCorp Baseline with AraBERT

This notebook is a sanity-check baseline for Arabic NER using **ANERCorp** and **AraBERT** (`aubmindlab/bert-base-arabertv02`).

We use it to validate that our token-classification setup is healthy before adapting the pipeline to Islamic-domain NER.


## What is ANERCorp and why use it?

ANERCorp is a standard Arabic NER benchmark with BIO labels for `PER`, `ORG`, `LOC`, and `MISC`.

We use it here because it is a fast sanity check for:
- data parsing,
- token/label alignment,
- training/evaluation wiring.

Typical AraBERT results are often around the **80-90 F1** range (depending on split/setup).


In [1]:
from __future__ import annotations

import importlib
import subprocess
import sys


def ensure_package(import_name: str, pip_name: str | None = None) -> None:
    try:
        importlib.import_module(import_name)
        return
    except ImportError:
        pkg = pip_name or import_name
        print(f"Installing missing package: {pkg} (kernel python: {sys.executable})")
        subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])


# Install into the active notebook kernel interpreter if missing.
ensure_package("evaluate")
ensure_package("seqeval")
ensure_package("datasets")
ensure_package("transformers")
ensure_package("accelerate")

import random
import re
import zipfile
from collections import Counter
from pathlib import Path
from typing import Dict, List, Tuple
from urllib.parse import urljoin

import evaluate
import numpy as np
import pandas as pd
import requests
from datasets import Dataset, DatasetDict, load_dataset
from seqeval.metrics import classification_report, f1_score, precision_score, recall_score
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    DataCollatorForTokenClassification,
    Trainer,
    TrainingArguments,
)

# Optional if environment is missing packages:
# !pip install -q transformers datasets evaluate seqeval accelerate

ROOT = Path.cwd().resolve()
if not (ROOT / "src").exists():
    ROOT = ROOT.parent.resolve()

DATA_DIR = ROOT / "data" / "raw" / "anercorp"
DATA_DIR.mkdir(parents=True, exist_ok=True)

MODEL_NAME = "aubmindlab/bert-base-arabertv02"
OUTPUT_DIR = ROOT / "models" / "anercorp_baseline"
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

print("Project root:", ROOT)
print("ANERCorp dir:", DATA_DIR)


Project root: C:\Users\diaab\islamic-ner
ANERCorp dir: C:\Users\diaab\islamic-ner\data\raw\anercorp


## 1) Download ANERCorp (HF first, known-source fallback)

We try HuggingFace first with `datasets.load_dataset("anercorp")` and similar names.
If that fails, we try known sources and parse CoNLL files (`word<TAB>tag`, blank line between sentences).


In [2]:
HF_DATASET_CANDIDATES = [
    "anercorp",
    "ANERcorp",
    "asas-ai/ANERCorp",
    "MagedSaeed/ANERcorp_experimental",
]

KNOWN_SOURCE_PAGES = [
    "https://camel.abudhabi.nyu.edu/anercorp/",
    "https://docs.google.com/forms/d/e/1FAIpQLSc4lNiI8eqjrsVQ0GBhgkctLCdLzBGshbdVWUogci3-wqnfoQ/viewform?usp=sf_link",
]

KNOWN_DIRECT_FILE_URLS = [
    # Add direct mirrors if needed.
]


def normalize_tag(tag: str) -> str:
    tag = str(tag).strip()
    if tag in {"", "0"}:
        return "O"
    tag = tag.replace("B-PERS", "B-PER").replace("I-PERS", "I-PER")
    return tag


def parse_conll_text(conll_text: str) -> List[Dict[str, List[str]]]:
    sentences: List[Dict[str, List[str]]] = []
    tokens: List[str] = []
    tags: List[str] = []

    for raw_line in conll_text.splitlines():
        line = raw_line.strip()
        if not line:
            if tokens:
                sentences.append({"tokens": tokens, "ner_tags": tags})
                tokens, tags = [], []
            continue

        parts = re.split(r"\s+", line)
        if len(parts) < 2:
            continue

        tokens.append(parts[0])
        tags.append(normalize_tag(parts[-1]))

    if tokens:
        sentences.append({"tokens": tokens, "ner_tags": tags})

    return sentences


def parse_conll_file(path: Path) -> List[Dict[str, List[str]]]:
    return parse_conll_text(path.read_text(encoding="utf-8", errors="replace"))


def maybe_load_hf_sentence_level(split) -> List[Dict[str, List[str]]]:
    records: List[Dict[str, List[str]]] = []
    if len(split) == 0:
        return records

    cols = set(split.column_names)
    row0 = split[0]

    token_key = next((k for k in ["tokens", "words", "word"] if k in cols), None)
    tag_key = next((k for k in ["ner_tags", "tags", "tag", "labels"] if k in cols), None)
    if token_key is None or tag_key is None:
        return records

    # Sentence-level already.
    if isinstance(row0[token_key], list):
        for row in split:
            tokens = [str(x) for x in row[token_key]]
            tags = [normalize_tag(x) for x in row[tag_key]]
            if tokens and len(tokens) == len(tags):
                records.append({"tokens": tokens, "ner_tags": tags})
        return records

    # Token-level with sentence id.
    sent_id_key = next(
        (k for k in ["sentence_id", "sent_id", "sid", "sentence", "doc_id"] if k in cols),
        None,
    )
    if sent_id_key is not None:
        grouped: Dict[str, Dict[str, List[str]]] = {}
        order: List[str] = []
        for row in split:
            sid = str(row[sent_id_key])
            if sid not in grouped:
                grouped[sid] = {"tokens": [], "ner_tags": []}
                order.append(sid)
            grouped[sid]["tokens"].append(str(row[token_key]))
            grouped[sid]["ner_tags"].append(normalize_tag(row[tag_key]))
        for sid in order:
            if grouped[sid]["tokens"]:
                records.append(grouped[sid])
        return records

    # Token-level without boundaries -> unsupported.
    return []


def convert_hf_to_dataset_dict(raw) -> DatasetDict | None:
    if isinstance(raw, DatasetDict):
        converted: Dict[str, Dataset] = {}
        for split_name in raw.keys():
            recs = maybe_load_hf_sentence_level(raw[split_name])
            if recs:
                converted[split_name] = Dataset.from_list(recs)
        return DatasetDict(converted) if converted else None

    recs = maybe_load_hf_sentence_level(raw)
    if not recs:
        return None
    return DatasetDict({"train": Dataset.from_list(recs)})


def try_load_from_hf() -> Tuple[DatasetDict | None, str | None, List[str]]:
    errors: List[str] = []
    for dataset_name in HF_DATASET_CANDIDATES:
        try:
            raw = load_dataset(dataset_name)
            ds = convert_hf_to_dataset_dict(raw)
            if ds is not None:
                return ds, dataset_name, errors
            errors.append(f"{dataset_name}: unsupported format")
        except Exception as exc:
            errors.append(f"{dataset_name}: {type(exc).__name__}: {exc}")
    return None, None, errors


def download_file(url: str, destination: Path) -> bool:
    destination.parent.mkdir(parents=True, exist_ok=True)
    try:
        with requests.get(url, stream=True, timeout=60) as response:
            response.raise_for_status()
            with destination.open("wb") as out:
                for chunk in response.iter_content(chunk_size=1024 * 64):
                    if chunk:
                        out.write(chunk)
        return True
    except Exception as exc:
        print(f"[skip] {url} -> {exc}")
        return False


def extract_links(html: str, base_url: str) -> List[str]:
    hrefs = re.findall(r'href=["\']([^"\']+)["\']', html)
    links = [urljoin(base_url, href) for href in hrefs]
    return list(dict.fromkeys(links))


def try_known_source_download() -> DatasetDict | None:
    local_train = [
        DATA_DIR / "train.txt",
        DATA_DIR / "anercorp_train.txt",
        DATA_DIR / "train.conll",
    ]
    local_test = [
        DATA_DIR / "test.txt",
        DATA_DIR / "anercorp_test.txt",
        DATA_DIR / "test.conll",
    ]

    train_path = next((p for p in local_train if p.exists()), None)
    test_path = next((p for p in local_test if p.exists()), None)
    if train_path and test_path:
        train_records = parse_conll_file(train_path)
        test_records = parse_conll_file(test_path)
        if train_records and test_records:
            return DatasetDict({
                "train": Dataset.from_list(train_records),
                "test": Dataset.from_list(test_records),
            })

    downloaded: List[Path] = []

    for url in KNOWN_DIRECT_FILE_URLS:
        name = url.split("?")[0].split("/")[-1] or "anercorp_file"
        out = DATA_DIR / name
        if download_file(url, out):
            downloaded.append(out)

    for page_url in KNOWN_SOURCE_PAGES:
        try:
            html = requests.get(page_url, timeout=30).text
        except Exception as exc:
            print(f"[skip-page] {page_url} -> {exc}")
            continue

        for link in extract_links(html, page_url):
            low = link.lower()
            if not (low.endswith(".txt") or low.endswith(".conll") or low.endswith(".zip") or "download" in low):
                continue
            name = link.split("?")[0].split("/")[-1] or "anercorp_download"
            out = DATA_DIR / name
            if download_file(link, out):
                downloaded.append(out)

    train_records: List[Dict[str, List[str]]] = []
    test_records: List[Dict[str, List[str]]] = []

    for path in downloaded:
        if path.suffix.lower() in {".txt", ".conll"}:
            records = parse_conll_file(path)
            name = path.name.lower()
            if "train" in name:
                train_records.extend(records)
            elif "test" in name:
                test_records.extend(records)
        elif path.suffix.lower() == ".zip":
            extract_dir = DATA_DIR / f"{path.stem}_extracted"
            extract_dir.mkdir(parents=True, exist_ok=True)
            try:
                with zipfile.ZipFile(path, "r") as zf:
                    zf.extractall(extract_dir)
                for txt in extract_dir.rglob("*.txt"):
                    records = parse_conll_file(txt)
                    name = txt.name.lower()
                    if "train" in name:
                        train_records.extend(records)
                    elif "test" in name:
                        test_records.extend(records)
            except Exception as exc:
                print(f"[skip-zip] {path} -> {exc}")

    if train_records and test_records:
        return DatasetDict({
            "train": Dataset.from_list(train_records),
            "test": Dataset.from_list(test_records),
        })

    return None


dataset_dict, hf_source_name, hf_errors = try_load_from_hf()

if dataset_dict is not None:
    print(f"Loaded from HuggingFace source: {hf_source_name}")
else:
    print("HF loading failed, trying known-source fallback...")
    for err in hf_errors:
        print(" -", err)
    dataset_dict = try_known_source_download()

if dataset_dict is None:
    raise RuntimeError(
        "Could not load ANERCorp from HF or known-source fallback. "
        "Place train/test CoNLL files in data/raw/anercorp/ and retry."
    )

print(dataset_dict)
print("Split sizes:", {k: len(v) for k, v in dataset_dict.items()})


Loaded from HuggingFace source: MagedSaeed/ANERcorp_experimental
DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3972
    })
    test: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 924
    })
})
Split sizes: {'train': 3972, 'test': 924}


## 2) Dataset exploration

We inspect:
- number of sentences and tokens,
- entity type distribution,
- random annotated sentence samples.


In [3]:
if "test" not in dataset_dict:
    split = dataset_dict["train"].train_test_split(test_size=0.2, seed=SEED)
    dataset_dict = DatasetDict({"train": split["train"], "test": split["test"]})

print("Using splits:", {k: len(v) for k, v in dataset_dict.items()})

all_rows = list(dataset_dict["train"]) + list(dataset_dict["test"])
sentence_count = len(all_rows)
token_count = sum(len(row["tokens"]) for row in all_rows)

print("Sentences:", sentence_count)
print("Tokens:", token_count)

entity_counter = Counter()
for row in all_rows:
    for tag in row["ner_tags"]:
        if tag == "O":
            continue
        ent = tag.split("-", 1)[1] if "-" in tag else tag
        entity_counter[ent] += 1

dist_df = pd.DataFrame([
    {"entity_type": ent, "count": entity_counter.get(ent, 0)}
    for ent in ["PER", "ORG", "LOC", "MISC"]
])
display(dist_df)

print("Sample annotated sentences:")
for row in random.sample(all_rows, k=min(5, len(all_rows))):
    pairs = [f"{tok}/{tag}" for tok, tag in list(zip(row["tokens"], row["ner_tags"]))[:35]]
    print(" ".join(pairs))
    print("-" * 80)


Using splits: {'train': 3972, 'test': 924}
Sentences: 4896
Tokens: 150072


,entity_type,count
0,PER,6421
1,ORG,3416
2,LOC,5052
3,MISC,1662


Sample annotated sentences:
وفي/O ضوء/O مجمل/O آرائه/O حولها/O ،/O من/O الصعب/O اعتباره/O عدوا/O ايديولوجيا/O لها/O مثل/O العقاد/B-PER ،/O فهو/O في/O الواقع/O صديق/O لها/O ،/O أو/O نصير/O ،/O وان/O جاز/O في/O هذا/O الاطار/O استدعاء/O العبارة/O التراثية/O المأثورة/O :/O '/O
--------------------------------------------------------------------------------
وأضاف/O أن/O العلاقات/O البحرينية/O ـ/O المصرية/O وطيدة/O ومتميزة/O في/O جميع/O المجالات/O ,/O وأشار/O إلي/O أن/O قمته/O مع/O الرئيس/O مبارك/B-PER تناولت/O التعاون/O بين/O البلدين/O الشقيقين/O ,/O واستعراض/O ما/O تمر/O به/O المنطقة/O العربية/O من/O تطورات/O سواء/O علي/O
--------------------------------------------------------------------------------
وفي/O تبريره/O للعملية/O ;/O قال/O الجيش/O الإسرائيلي/O إنه/O نفذها/O بهدف/O منع/O تهريب/O أسلحة/O من/O سوريا/B-LOC وإيران/B-LOC إلي/O حزب/B-ORG الله/I-ORG ./O
--------------------------------------------------------------------------------
ومن/O الشواهد/O التي/O نعتقد/O أن/O أصحابها/O قد/O ي

## 3) Fine-tune AraBERT for token classification

### Subword-label alignment (why needed)

BERT tokenizers split some words into multiple subword pieces.
NER labels are word-level, so we align as follows:
- first subword gets the BIO label,
- remaining subwords get `-100` so the loss ignores them.


In [4]:
label_list = sorted({tag for row in dataset_dict["train"] for tag in row["ner_tags"]})
if "O" in label_list:
    label_list = ["O"] + [label for label in label_list if label != "O"]

label2id = {label: i for i, label in enumerate(label_list)}
id2label = {i: label for label, i in label2id.items()}

print("Labels:", label_list)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)


def tokenize_and_align_labels(examples):
    tokenized = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=256,
    )

    aligned = []
    for i in range(len(examples["tokens"])):
        word_ids = tokenized.word_ids(batch_index=i)
        word_tags = examples["ner_tags"][i]

        prev_word = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != prev_word:
                label_ids.append(label2id[word_tags[word_idx]])
            else:
                label_ids.append(-100)
            prev_word = word_idx

        aligned.append(label_ids)

    tokenized["labels"] = aligned
    return tokenized


tokenized_datasets = dataset_dict.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=dataset_dict["train"].column_names,
    desc="Tokenize + align labels",
)

tokenized_datasets


Labels: ['O', 'B-LOC', 'B-MISC', 'B-ORG', 'B-PER', 'I-LOC', 'I-MISC', 'I-ORG', 'I-PER']


Tokenize + align labels:   0%|          | 0/3972 [00:00<?, ? examples/s]

Tokenize + align labels:   0%|          | 0/924 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 3972
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 924
    })
})

In [5]:
seqeval_metric = evaluate.load("seqeval")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    pred_ids = np.argmax(logits, axis=2)

    y_true = []
    y_pred = []

    for pred_row, label_row in zip(pred_ids, labels):
        row_true = []
        row_pred = []
        for pred_id, label_id in zip(pred_row, label_row):
            if label_id == -100:
                continue
            row_true.append(id2label[int(label_id)])
            row_pred.append(id2label[int(pred_id)])
        y_true.append(row_true)
        y_pred.append(row_pred)

    scores = seqeval_metric.compute(predictions=y_pred, references=y_true)
    return {
        "precision": scores["overall_precision"],
        "recall": scores["overall_recall"],
        "f1": scores["overall_f1"],
        "accuracy": scores["overall_accuracy"],
    }


model = AutoModelForTokenClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(label_list),
    id2label=id2label,
    label2id=label2id,
)

training_args = TrainingArguments(
    output_dir=str(ROOT / "models" / "anercorp_baseline_runs"),
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_steps=50,
    seed=SEED,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForTokenClassification(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.evaluate()


Some weights of BertForTokenClassification were not initialized from the model checkpoint at aubmindlab/bert-base-arabertv02 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
C:\Users\diaab\AppData\Roaming\Python\Python312\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/747 [00:00<?, ?it/s]

C:\Users\diaab\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


{'loss': 0.388, 'grad_norm': 0.6761423349380493, 'learning_rate': 2.7991967871485944e-05, 'epoch': 0.2}
{'loss': 0.1061, 'grad_norm': 1.1202919483184814, 'learning_rate': 2.598393574297189e-05, 'epoch': 0.4}
{'loss': 0.0756, 'grad_norm': 0.9610965847969055, 'learning_rate': 2.397590361445783e-05, 'epoch': 0.6}
{'loss': 0.0682, 'grad_norm': 0.5251806974411011, 'learning_rate': 2.1967871485943775e-05, 'epoch': 0.8}


  0%|          | 0/58 [00:00<?, ?it/s]

...

  0%|          | 0/58 [00:00<?, ?it/s]

{'eval_loss': 0.1441417932510376, 'eval_precision': 0.8406320541760722, 'eval_recall': 0.8152364273204904, 'eval_f1': 0.8277394976661481, 'eval_accuracy': 0.9709553528564571, 'eval_runtime': 47.2043, 'eval_samples_per_second': 19.574, 'eval_steps_per_second': 1.229, 'epoch': 3.0}
{'train_runtime': 3590.3367, 'train_samples_per_second': 3.319, 'train_steps_per_second': 0.208, 'train_loss': 0.07206859151362735, 'epoch': 3.0}


C:\Users\diaab\AppData\Roaming\Python\Python312\site-packages\torch\utils\data\dataloader.py:775: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  super().__init__(loader)


  0%|          | 0/58 [00:00<?, ?it/s]

{'eval_loss': 0.1441417932510376,
 'eval_precision': 0.8406320541760722,
 'eval_recall': 0.8152364273204904,
 'eval_f1': 0.8277394976661481,
 'eval_accuracy': 0.9709553528564571,
 'eval_runtime': 47.0213,
 'eval_samples_per_second': 19.651,
 'eval_steps_per_second': 1.233,
 'epoch': 3.0}

## 4) Evaluate with seqeval

We print per-entity precision/recall/F1, overall F1, and the full classification report.


In [6]:
pred_output = trainer.predict(tokenized_datasets["test"])
pred_ids = np.argmax(pred_output.predictions, axis=2)
label_ids = pred_output.label_ids

y_true = []
y_pred = []
for pred_row, label_row in zip(pred_ids, label_ids):
    row_true = []
    row_pred = []
    for p, l in zip(pred_row, label_row):
        if l == -100:
            continue
        row_true.append(id2label[int(l)])
        row_pred.append(id2label[int(p)])
    y_true.append(row_true)
    y_pred.append(row_pred)

overall_p = precision_score(y_true, y_pred)
overall_r = recall_score(y_true, y_pred)
overall_f1 = f1_score(y_true, y_pred)

print(f"Overall Precision: {overall_p:.4f}")
print(f"Overall Recall:    {overall_r:.4f}")
print(f"Overall F1:        {overall_f1:.4f}")
print()
print("Classification report:")
print(classification_report(y_true, y_pred, digits=4))

report_dict = classification_report(y_true, y_pred, output_dict=True, digits=4)
per_entity_rows = []
for entity in ["PER", "ORG", "LOC", "MISC"]:
    if entity in report_dict:
        per_entity_rows.append({
            "entity": entity,
            "precision": report_dict[entity]["precision"],
            "recall": report_dict[entity]["recall"],
            "f1": report_dict[entity]["f1-score"],
            "support": report_dict[entity]["support"],
        })

if per_entity_rows:
    display(pd.DataFrame(per_entity_rows))


  0%|          | 0/58 [00:00<?, ?it/s]

Overall Precision: 0.8406
Overall Recall:    0.8152
Overall F1:        0.8277

Classification report:
              precision    recall  f1-score   support

         LOC     0.8881    0.9275    0.9074       676
        MISC     0.6798    0.5679    0.6188       243
         ORG     0.7596    0.7364    0.7478       459
         PER     0.8815    0.8377    0.8591       906

   micro avg     0.8406    0.8152    0.8277      2284
   macro avg     0.8022    0.7674    0.7833      2284
weighted avg     0.8375    0.8152    0.8255      2284



,entity,precision,recall,f1,support
0,PER,0.881533,0.837748,0.859083,906
1,ORG,0.759551,0.736383,0.747788,459
2,LOC,0.888102,0.927515,0.907381,676
3,MISC,0.679803,0.567901,0.618834,243


## 5) Save trained model

This saves the sanity-check baseline model for quick reuse.


In [7]:
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
trainer.save_model(str(OUTPUT_DIR))
tokenizer.save_pretrained(str(OUTPUT_DIR))

print(f"Saved baseline model to: {OUTPUT_DIR}")


Saved baseline model to: C:\Users\diaab\islamic-ner\models\anercorp_baseline


## What results tell us and next adaptation step

If the setup is healthy, AraBERT on ANERCorp is commonly around **80-90 F1**.

If lower than expected, check:
- parsing quality,
- label alignment,
- train/test split consistency.

Next we adapt this pipeline for Islamic NER by replacing labels/data and adding weak supervision from gazetteers + rules.
